In [1]:
import edgar
import os, time
from pathlib2 import Path
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import ProjectDirectory as directory
pd.options.mode.chained_assignment = None

## generate df with all companies and URLs

In [2]:
project_dir = directory.get_project_dir()

In [3]:
os.chdir(os.path.join(project_dir, 'sec-filings-index'))

In [4]:
# filing_year = 2013   # uncomment to run, choose year to get all edgar filings from
# edgar.download_index(os.getcwd(), filing_year)

In [5]:
# Get list of all DFs 
table_list = []

for i in os.listdir():
    if i.endswith('.tsv'):
        table_list.append(pd.read_csv(i, sep='|', header=None, encoding='latin-1', parse_dates=[3], dtype={0: int}))

In [6]:
# append all dfs into a single df
df = pd.DataFrame(columns=[0,1,2,3,4,5])   # downloaded file has 6 columns

for i in range(len(table_list)):
        df = pd.concat([df, table_list[i]], ignore_index=True, axis=0)

df.columns= ['cik', 'company_name', 'filing_type', 'filing_date', 'url', 'url2']

## Check if dataframe correctly generated

In [7]:
count_list = []
for i in range(len(table_list)):
    count_list.append(len(table_list[i]))

if df.shape[0] == sum(count_list):
    print('df tallies with individual files. Total rows = {}'.format(df.shape[0]))
else:
    print('ERROR. df does not tally!!')

df tallies with individual files. Total rows = 6061186


## Get CIK df

In [8]:
# cik_ticker_list.csv contains cik tickets of companies
df_cik = pd.read_csv(os.path.join(project_dir, 'data', 'cik_ticker_list.csv'))

In [9]:
def company_name_search(df, company_name_list):
    for company in company_name_list:
        df_company = df[df['Name'].str.contains(company, case=False)]
        print('*' * 50)
        print('SEARCH TERM: {}'.format(company))
        print('RESULTS:')
        for i in df_company['Name'].tolist():
            for j in df_company['CIK'].tolist():
                print(i, j)
        print('*' * 50)
        

In [10]:
def get_cik_from_company_name(df, company_name_list):
    cik_list = []
    for company in company_name_list:
        cik_series = df[df['Name'].str.contains(company, case=False)]['CIK']
        cik_list.append(cik_series.values[0])
    return cik_list

In [11]:
def get_company_name_from_cik(df, cik_list):
    company_list = []
    for cik in cik_list:
        company_series = df[df['CIK'] == cik]
        company_list.append(company_series.values[0])
    return company_list

In [12]:
companies_list = ['apple inc', 'tesla', 'netflix', 
                  'amazon com inc', 'microsoft', 'facebook']

In [13]:
company_name_search(df_cik, companies_list)

**************************************************
SEARCH TERM: apple inc
RESULTS:
Apple Inc 320193
**************************************************
**************************************************
SEARCH TERM: tesla
RESULTS:
Tesla Motors Inc 1318605
**************************************************
**************************************************
SEARCH TERM: netflix
RESULTS:
Netflix Inc 1065280
**************************************************
**************************************************
SEARCH TERM: amazon com inc
RESULTS:
Amazon Com Inc 1018724
**************************************************
**************************************************
SEARCH TERM: microsoft
RESULTS:
Microsoft Corp 789019
**************************************************
**************************************************
SEARCH TERM: facebook
RESULTS:
Facebook Inc 1326801
**************************************************


In [14]:
cik_list = get_cik_from_company_name(df_cik, companies_list)

## download data

In [15]:
def download_filings(cik_num_list, from_date='2014-01-01'):
    """Function to filter the appropriate filings and download them in the folder"""
    
    project_dir = directory.get_project_dir()
    
    # filter df with company CIK,filing type (10-K and 10-Q) and date  
    df_filtered = df [(df['cik'].isin(cik_num_list)) & 
                      ((df['filing_type']=='10-K') | (df['filing_type'] == '10-Q')) & 
                      (df['filing_date'] > from_date)]
    
    company_names = df_filtered['company_name'].unique().tolist()
    
    # check if folders for each company already exists    
    sec_filings_dir = os.path.join(project_dir, 'sec-filings-downloaded')  # dir to download SEC filingsa
    os.chdir(sec_filings_dir)

    for company in company_names:
        company_dir = os.path.join(sec_filings_dir, company)

        if not os.path.exists(company_dir):
            os.makedirs(company_dir)
            print('\n created dir: {}'.format(company))
        else:
            print('\n{} directory exists'.format(company))
            
        os.chdir(company_dir)
        
        # create company specific df to iterate over    
        df_filtered_co = df_filtered[df_filtered['company_name'] == company]  # get df with the company only
        df_filtered_co['filing_date'] = df_filtered_co['filing_date'].astype(str)   # convert to 'object' to name file
        
        for i in range(len(df_filtered_co)):
            url_prefix = 'https://www.sec.gov/Archives/'
            row = df_filtered_co.iloc[i,:]
            url = url_prefix + row['url']
            response = requests.get(url, stream=True, timeout=30)
            
            filing_name = row['filing_date'] + str('_') + row['filing_type']
            if os.path.isfile(filing_name):
                print('{} file already exists'.format(filing_name))
            else:
                print('Downloading: {}'.format(filing_name))
                with open('{}'.format(filing_name), 'wb') as handle:
                    for data in tqdm(response.iter_content()):
                        handle.write(data)    

### ↓ Automated download of filings. If the filing exists in the directory, the download will skip and move on the the next filing

In [16]:
download_filings(cik_list)


 created dir: AMAZON COM INC
Downloading: 2018-07-27_10-Q


522285it [00:07, 74024.13it/s] 


Downloading: 2019-02-01_10-K


865310it [00:13, 63133.86it/s]


Downloading: 2018-02-02_10-K


...

502885it [00:03, 141578.46it/s]


Downloading: 2014-05-09_10-Q


519382it [00:03, 149754.04it/s]


Downloading: 2014-02-26_10-K


977196it [00:06, 146324.26it/s]
